

*   будем показывать случайные баннеры на любой клик,
который придёт. Наша модель состоит из двух стадий: отбор кандидатов (те офферы, на которые пользователь физически может перейти и подписаться, в зависимости от его географии, устройства, источника и т.д.) и выбор оффера (то, что мы будем реализовывать).
*   Кандидатная модель уже реализована.В наш ML-сервис поступает сразу клик (click ID) и набор офферов-кандидатов (offer IDs). Мы хотим брать случайный из них. Для тестирования интеграции с бекендом этого более чем достаточно.

In [ ]:
import numpy as np
import uvicorn
from fastapi import FastAPI
import random

app = FastAPI()

@app.get("/sample/")
def sample(offer_ids: str) -> dict:
    """Sample random offer"""
    # Parse offer IDs
    offers_ids = [int(offer) for offer in offer_ids.split(",")]

    # Sample random offer ID
    offer_id = random.choice(offers_ids)

    # Prepare response
    response = {
        "offer_id": offer_id,
    }
    return response

def main() -> None:
    """Run application"""
    uvicorn.run("app:app", host="localhost", port=8000, reload=False)

if __name__ == "__main__":
    main()


Давайте теперь реализуем более сложную версию, которая берёт во внимание «обратную связь» (feedback) от пользователя и на каждый клик выдаёт оффер, максимизирующий выручку (жадный алгоритм).

**Feedback**

Первая задаёт **PUT-запрос /feedback**. PUT-запрос используется, когда в сервис отправляется какая-то информация извне (в отличие от GET-запроса с предыдущего шага, который **получает какую-то информацию от сервиса**).

/feedback отдаёт информацию о том, что случилось с кликом, сколько денег (в центах) мы за него получили. Если конверсии не было, то **reward=0** (напомним, что конверсия — это когда пользователь перешёл на оффер и выполнил нужное действие: скачал приложение, закинул деньги в казино и т.д.).

In [ ]:
from fastapi import FastAPI

app = FastAPI()
# GLOBAL DICTIONARIES (reset before each run)
click_to_offer: Dict[int, int] = dict()      # click_id -> offer_id
offer_clicks: Dict[int, int] = dict()        # offer_id -> clicks count
offer_rewards: Dict[int, float] = dict()     # offer_id -> total reward
offer_conversions: Dict[int, int] = dict()   # offer_id -> conversion count
total_clicks: int = 0                        # Global click counter

In [ ]:
@app.put("/feedback/")
def feedback(click_id: int, reward: float) -> dict:
    """Get feedback for particular click"""
    #click_to_offer [click_id по какому_то конкретному offer_id, т.е 101 click перешел в offer]
    offer_id = click_to_offer.get(click_id,None)
    is_conversion = reward > 0
    # Response body consists of click ID
    # and accepted click status (True/False)
    if offer_id is not None:
      if is_conversion:
        offer_conversions[offer_id] = offer_conversions.get(offer_id,0)+ 1 #-> converstions count
      offer_rewards[offer_id] = offer_rewards.get(offer_id,0.0) # -> converstions sum count
    else:
      pass

    response = {
        "click_id": click_id,
        "offer_id": offer_id,
        "is_conversion": is_conversion,
        "reward": reward,
    }
    return response

**Stats**
Ещё один GET-запрос, который вам необходимо реализовать – это /stats с актуальной информацией по выбранному офферу о том, какие были совершены клики, какие получены конверсии, какое среднее число денег мы получаем за клик и т.д.

*   RPC (revenue per click) – средняя выручка на клик

In [ ]:
@app.get("/offer_ids/{offer_id}/stats/")
def stats(offer_id: int) -> dict:
    """Return offer's statistics"""
    clicks = offer_clicks.get(offer_id,0)
    converstions = offer_converstions.get(offer_id,0)
    reward = offer_rewards(offer_id, 0.0)
    cr = (converstions/clicks) if clicks > 0 else 0.0
    rpc = (reward / clicks) if clicks > 0 else 0.0

    response = {
        "offer_id": offer_id,
        "clicks": 10,
        "conversions": 1,
        "reward": 4.2,
        "cr": 0.1,
        "rpc": 4.2 / 10,
    }
    return response

Наконец, вам необходимо переписать /sample так, чтобы:

На **первые 100 кликов**, оправленные в сервис, выбирался случайный (для инициализации)
На последующие выбирался тот (среди баннеров-кандидатов), который максимизирует **RPC.** Выбирать необходимо среди баннеров-кандидатов, переданных в аргументе offer_ids
Если не нашли подходящего оффера, возвращаем самый первый. Например, на вход пришли офферы offers_ids: [45, 67]. Но статистика по обоим офферам нулевая. Тогда выбираем 45

- Интернет-магазин хочет больше покупателей. Они обращаются в партнёрку.
- Вася знает, как сделать интересную рекламу в соцсетях. Он регистрируется в этой партнёрке.
- Вася размещает свою рекламу, люди по ней переходят и покупают в интернет-магазине.
- Магазин платит комиссию партнёрке, партнёрка — Васе.

In [ ]:
from fastapi import FastAPI
import numpy as np
import uvicorn
from typing import Dict
app = FastAPI()

# Глобальные переменные для хранения статистики
clicks_count = {}  # количество кликов по каждому офферу
conversions_count = {}  # количество конверсий по каждому офферу
total_rewards = {}  # суммарная награда по каждому офферу
click_to_offer = {}  # связь между click_id и offer_id
total_clicks = 0  # общее количество кликов

@app.on_event("startup")
def startup_event():
    """Сброс статистики при старте сервиса"""
    global clicks_count, conversions_count, total_rewards, click_to_offer, total_clicks
    clicks_count = {}
    conversions_count = {}
    total_rewards = {}
    click_to_offer = {}
    total_clicks = 0

@app.put("/feedback/")
def feedback(click_id: int, reward: float) -> dict:
    """Get feedback for particular click"""
    global clicks_count, conversions_count, total_rewards

    if click_id not in click_to_offer:
        return {"error": "Click ID not found"}

    offer_id = click_to_offer[click_id]
    is_conversion = reward > 0

    # Обновляем статистику
    if is_conversion:
        conversions_count[offer_id] = conversions_count.get(offer_id, 0) + 1
        total_rewards[offer_id] = total_rewards.get(offer_id, 0) + reward

    response = {
        "click_id": click_id,
        "offer_id": offer_id,
        "is_conversion": is_conversion,
        "reward": reward if is_conversion else 0
    }
    return response

@app.get("/offer_ids/{offer_id}/stats/")
def stats(offer_id: int) -> dict:
    """Return offer's statistics"""
    clicks = clicks_count.get(offer_id, 0)
    conversions = conversions_count.get(offer_id, 0)
    reward = total_rewards.get(offer_id, 0)

    cr = conversions / clicks if clicks > 0 else 0
    rpc = reward / clicks if clicks > 0 else 0

    response = {
        "offer_id": offer_id,
        "clicks": clicks,
        "conversions": conversions,
        "reward": reward,
        "cr": cr,
        "rpc": rpc,
    }
    return response

@app.get("/sample/")
def sample(click_id: int, offer_ids: str) -> dict:
    """Greedy sampling"""
    global total_clicks, click_to_offer, clicks_count

    # Parse offer IDs
    offers_ids = [int(offer) for offer in offer_ids.split(",")]

    # Выбираем оффер
    if total_clicks < 100:
        # Первые 100 кликов - случайный выбор
        import random
        offer_id = random.choice(offers_ids)
        sampler = "random"
    else:
        # Greedy выбор - оффер с максимальным RPC
        max_rpc = -1
        offer_id = offers_ids[0]  # дефолтный выбор

        for oid in offers_ids:
            stats_data = stats(oid)
            current_rpc = stats_data["rpc"]

            if current_rpc > max_rpc:
                max_rpc = current_rpc
                offer_id = oid

        sampler = "greedy"

    # Обновляем статистику
    clicks_count[offer_id] = clicks_count.get(offer_id, 0) + 1
    click_to_offer[click_id] = offer_id
    total_clicks += 1

    # Prepare response
    response = {
        "click_id": click_id,
        "offer_id": offer_id,
        "sampler": sampler,
    }
    return response

В предыдущих двух шагах вы поочерёдно реализовали две крайности:

1. Exploration: исследование офферов (случайное сэмплирование)
2. Exploitation: выбор лучшего оффера (жадное сэмплирование)



Мы с вами реализуем простейший алгоритм многоруких бандитов, называемый epsilon-greedy, который совмещает лучшее из двух миров. С вероятностью **1-ε** он будет максимизировать Exploitation, выбирая оффер, максимизирующий выручку, и с вероятностью **ε** – случайный из предложенных. **-многорукие бандиты**

In [ ]:
from fastapi import FastAPI
import numpy as np
import uvicorn
import random
from typing import Dict
app = FastAPI()

# Глобальные переменные для хранения статистики
clicks_count = {}  # количество кликов по каждому офферу
conversions_count = {}  # количество конверсий по каждому офферу
total_rewards = {}  # суммарная награда по каждому офферу
click_to_offer = {}  # связь между click_id и offer_id
total_clicks = 0  # общее количество кликов

@app.on_event("startup")
def startup_event():
    """Сброс статистики при старте сервиса"""
    global clicks_count, conversions_count, total_rewards, click_to_offer, total_clicks
    clicks_count = {}
    conversions_count = {}
    total_rewards = {}
    click_to_offer = {}
    total_clicks = 0

@app.put("/feedback/")
def feedback(click_id: int, reward: float) -> dict:
    """Get feedback for particular click"""
    global clicks_count, conversions_count, total_rewards

    if click_id not in click_to_offer:
        return {"error": "Click ID not found"}

    offer_id = click_to_offer[click_id]
    is_conversion = reward > 0

    # Обновляем статистику
    if is_conversion:
        conversions_count[offer_id] = conversions_count.get(offer_id, 0) + 1
        total_rewards[offer_id] = total_rewards.get(offer_id, 0) + reward

    response = {
        "click_id": click_id,
        "offer_id": offer_id,
        "is_conversion": is_conversion,
        "reward": reward if is_conversion else 0
    }
    return response

@app.get("/offer_ids/{offer_id}/stats/")
def stats(offer_id: int) -> dict:
    """Return offer's statistics"""
    clicks = clicks_count.get(offer_id, 0)
    conversions = conversions_count.get(offer_id, 0)
    reward = total_rewards.get(offer_id, 0)

    cr = conversions / clicks if clicks > 0 else 0
    rpc = reward / clicks if clicks > 0 else 0

    response = {
        "offer_id": offer_id,
        "clicks": clicks,
        "conversions": conversions,
        "reward": reward,
        "cr": cr,
        "rpc": rpc,
    }
    return response

@app.get("/sample/")
def sample(click_id: int, offer_ids: str) -> dict:
    """Epsilon-greedy sampling (epsilon=0.1)"""
    epsilon = 0.1
    global total_clicks, click_to_offer, clicks_count

    # Parse offer IDs
    offers_ids = [int(offer) for offer in offer_ids.split(",")]

    # Epsilon-greedy выбор
    if random.random() < epsilon:
        offer_id = random.choice(offers_ids)
        sampler = "random"
    else:
        max_rpc = -float('inf')
        offer_id = offers_ids[0]
        for oid in offers_ids:
            stats_data = stats(oid)
            current_rpc = stats_data["rpc"]
            if current_rpc > max_rpc:
                max_rpc = current_rpc
                offer_id = oid
        sampler = "greedy"

    # Обновляем статистику
    clicks_count[offer_id] = clicks_count.get(offer_id, 0) + 1
    click_to_offer[click_id] = offer_id
    total_clicks += 1

    response = {
        "click_id": click_id,
        "offer_id": offer_id,
        "sampler": sampler,
    }
    return response